# Example Notebook:
This notebook provides an example on how to use the OOINet download tool to perform the following functions:
* Search for datasets
* Identify desired reference designator
* Get the associated metadata for a given reference designator
* Request netCDF datasets for a reference designator
* Download the netCDF dataset to your local machine

The key parameters which the OOINet API requires is the "reference designator." A reference designator may be thought of as a type of instrument located at a fixed location and depth. For example, below use the **CP01CNSM-RID27-03-CTDBPC000**, which is the CTD located at 7 meters depth on the Pioneer Array Central Surface Mooring at approximately (latitude, longitude) of (40.14, -70.7783).

In [ ]:
# This is necessary if not installed as a package
import sys
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
sys.path.append("../ooinet/")

In [ ]:
# Import the M2M module
import M2M

---
## Search Datasets
First, we can search the available OOI Reference Designators (i.e. "refdes" for short) on the following keys: **array**, **node**, **instrument**. Additionally, can request for "**English_names**", which will return the descriptive name for the associated array, node, and instrument. Below, we will search for the available CTD instruments on the Pioneer Array Central Surface Mooring.

The major caveat with the search is, similar to searching on ERDDAP datasets, the search terms must be partial or full match based on OOI nomenclature. For example, we have to search for "CTD", "CTDBP", or the full instrument name "03-CTDBPC000". We can't search "conductivity", "temperature" or other CTD-related instrument terms.

In [ ]:
datasets = M2M.search_datasets(array="CP01CNSM", instrument="CTD", English_names=True)
datasets

From the above datasets, we're going to select the CTDBP instrument on the Pioneer Array Central Surface Mooring Near-Surface Instrument Frame (located at 7m depth), which has a reference designator **CP01CNSM-RID27-03-CTDBPC000**. 

In [ ]:
refdes = "CP01CNSM-RID27-03-CTDBPC000"

---
## Metadata
Next, we can query OOINet for the metadata associated with the selected reference designator. The metadata contains such valuable information such as the available methods and streams (which are required to download the data), the particleKeys (the data variable names), and the associated units. 

In [ ]:
metadata = M2M.get_metadata(refdes=refdes)
metadata

---
## Deployment Information
When we searched for CTD datasets on the Pioneer Central Surface Mooring, it returned a table which listed the available deployment numbers for each of the datasets. We can get much more detailed information on the deployments for a particular reference designator by requesting the deployment information from OOINet.

In [ ]:
deployments = M2M.get_deployments(refdes=refdes)
deployments

---
## Vocab Information
Additionally, if we are interested in more detailed information on the location that the reference designator is assigned to, we can request the vocab information for the given reference designator. The vocab information includes some of the "**English_names**" info we requested when searching for datasets, as well as instrument model, manufacturer, and the descriptive names for the reference designator location.

In [ ]:
vocab = M2M.get_vocab(refdes=refdes)
vocab

---
## Calibration Information
We can also request the calibration information for a given reference designator. Since individual instruments are swapped during each mooring deployment & recovery, the calibration coefficients for a reference designator are different for each deployment. The way OOI operates is that it loads all the available calibration coefficients for a given reference designator. Then, for each deployment, it finds the calibration coefficients with the most recent calibration date which most closely _precedes_ the start of the deployment. The result is a table, sorted by deployment number for a reference designator, with the uid of the specific instrument, its calibration coefficients, when the instrument was calibrated, and the source of the calibration coefficients.

In [ ]:
calibrations = M2M.get_calibrations_by_refdes(refdes, deployments)
calibrations

It is also possible to request the calibration history for a specific instrument by utilizing the **uid** of the instrument and using the ```get_api``` method and ```M2M.URLS``` attribute to construct your own request.

In [ ]:
uid = "CGINS-CTDBPC-50109" # This is unique to each instrument
instrument_cals = M2M.get_calibrations_by_uid(uid)
instrument_cals

---
## Download Datasets
The ultimate goal of the queries above were to identify what data streams(s) we are interested in requesting data from to download. Now we want to be able to request those data streams and get the associated netCDF files. This process involves the following steps:
1. Identify the methods and data streams for the selected reference designator
2. Request the THREDDS server url for the data sets
3. Get the catalog of datasets on the THREDDS server
4. Parse the catalog for the desired netCDF files
5. Download the identified netCDF files to a local directory

**1.** Get the methods and data streams associated with the given reference designator:

In [ ]:
streams = M2M.get_datastreams(refdes)
streams

**2.** Now, we request the THREDDS server url from OOINet. At a minimum, this requires the reference designator, method, and stream as inputs. This will request the datasets for _all_ deployments.

If we want to further limit the request to a specific deployment or a specific time period, we can do that by passing the arguments **beginDT** (begin datetime) and **endDT** (end datetime). 

Additionally, we can input some optional arguments that will return diagnostic information. The **include_provenance** will return a separate text file with information on the provenance of the data, such as the calibration coefficients applied. The **include_annotations** returns a separate text file of annotations, which are descriptions of issues and information associated with the given dataset.

In [ ]:
method = "recovered_inst"
stream = "ctdbp_cdef_instrument_recovered"

In [ ]:
thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)
thredds_url

**3.** With the appropriate THREDDS url, we can query the THREDDS catalog to get the netCDF datasets. Additionally, we'll clean the datasets to delete potentially empty datasets in the catalog (this is common for some instruments):

In [ ]:
catalog = M2M.get_thredds_catalog(thredds_url)
catalog

**4.** Next, we want to clean up the THREDDS catalog of either unwanted datasets or empty datasets. Depending on the instrument, it may generate a dataset file even if there was no data in the time period of the file. 

In [ ]:
catalog = M2M.clean_catalog(catalog, stream, deployments)
catalog

**5.** Finally, we can download the netCDF files to a specified save directory (**saveDir**). The available function utilizes multithreading to speed up I/O operations. 

In [ ]:
saveDir = "/home/areed/Documents/OOI/reedan88/ooinet/examples/data/"
M2M.download_netCDF_files(catalog, goldCopy=True, saveDir=saveDir)

---
## Open netCDF Datasets
If, instead of downloading the datasets to a local directory, we wanted to load those datasets into a single dataset remotely? This can be done using the ```load_netCDF_ddatasets``` method, which takes in the THREDDS catalog queried above and utilizes ```xarray.open_mfdataset``` to load the datasets into a single ```xarray.DataSet```. Additionally, the ```load_netCDF_datasets``` checks each file to make sure it is properly formed by identifying empty or malformed datasets.

However, there is a complication when opening multiple netCDF files as a single dataset. OOI makes use of overlapping deployments, such that the CTD for deployment 11 goes into the water and starts collecting data _before_ the CTD from deployment 10 comes out of the water. Unfortunately, ```xarray.open_mfdatasets``` disallows overlapping primary dimensions. Consequently, its suggested you utilizes a preprocessing routine to trim datasets based on deployment times to avoid overlapping time indices. In the above example, the trimming would cut off the end of the CTD record for deployment 10 in favor of keeping the start of the CTD record from deployment 11.

If you desire to retain the overlapping time periods, this may be accomplished by concatentating the datasets. Beware, concatentation requires loading _all_ of the data into memory and cannot take advantage of the built-in dask parallel processing, which significantly slows down dataset loading and, for large datasets (such as profilers) you are likely to run out of memory before the dataset is fully concatenated.